In [2]:
import QUANTAXIS as QA
import jupyter_helper
import talib
import numpy as np
import pyecharts
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
from talib import MA_Type



#加载 seaborn，并且设置默认使用 seaborn
import seaborn as sns
sns.set()
import matplotlib
matplotlib.rcParams['font.family'] = 'sans-serif'
matplotlib.rcParams['font.sans-serif'] = ['SimHei']
matplotlib.rcParams['axes.unicode_minus']=False #用来正常显示负号

In [3]:
s,e=jupyter_helper.get_start_end_date()
benchmark_code=jupyter_helper.get_benchmark_code()
stock_code='601398'
DAYS=1#计算时间段

In [4]:
calc_data=jupyter_helper.get_calc_data(stock_code,s,e)
# calc_data.head()

In [5]:
def create_validate_df_close(df, days,column='close') -> pd.DataFrame:
    '''制作结果数据。
    '''
    if column not in df.columns:
        raise ValueError('数据中不包含 {} 列。'.format(column))
    return pd.DataFrame(calc_data[column].shift(-1*days).values,index=calc_data.index).dropna()

In [65]:
def vv(days, test_size=0.2, column='close'):
    calc_data = jupyter_helper.get_calc_data(stock_code, s, e)

    y = create_validate_df_close(calc_data, days, column=column)
#     return calc_data,y
    
    X = calc_data[calc_data.index.isin(y.index)]
#     return X,y

    X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                        test_size=test_size,
                                                        random_state=1)
    
    ridge = Ridge()
    ridge.fit(X_train, y_train)
    print('score:{}'.format(ridge.score(X_test, y_test)))
#     print('实际值')
#     print(calc_data[-1*days:][column])
#     print('预测值')
#     print(np.round(ridge.predict(calc_data[-1*days:]),2))
    predict_value = np.round(ridge.predict(calc_data[-1*days:]), 2)
    
    return ridge,calc_data[-1*days:],pd.DataFrame(
        predict_value, columns=['{}_{}'.format(column, days)])
    
    result = calc_data[-1*days:][column].reset_index().join(pd.DataFrame(
        predict_value, columns=['{}_{}'.format(column, days)])).set_index('date')
#     if result.shape[0]>1:
#         result.plot(title='真实值/预测值')
#         result.pct_change().plot(title='变化率')
#     print(result)
    return result

In [66]:
f,t,v=vv(2)
t

score:0.9897215607353016


,open,high,low,close,volume,amount
date,,,,,,
2019-01-23,5.40,5.45,5.39,5.44,0.025082,0.038346
2019-01-24,5.46,5.47,5.42,5.46,0.016757,0.026535


In [64]:
v

,close_2
0,5.43
1,5.45


In [62]:
calc_data.tail()

,open,high,low,close,volume,amount
date,,,,,,
2019-01-18,5.27,5.40,5.25,5.39,0.053856,0.078348
2019-01-21,5.41,5.48,5.40,5.45,0.044941,0.066872
2019-01-22,5.46,5.46,5.37,5.39,0.028306,0.042808
2019-01-23,5.40,5.45,5.39,5.44,0.025082,0.038346
2019-01-24,5.46,5.47,5.42,5.46,0.016757,0.026535


In [67]:
calc_data

,open,high,low,close,volume,amount
date,,,,,,
2006-10-27,1.992133,2.015570,1.910104,1.921823,1.000000,0.521280
2006-10-30,1.915963,1.945260,1.904245,1.927682,0.133277,0.066700
2006-10-31,1.921823,1.951119,1.921823,1.933541,0.085953,0.043163
2006-11-01,1.933541,1.939400,1.921823,1.933541,0.048172,0.023725
2006-11-02,1.933541,1.933541,1.904245,1.921823,0.064593,0.031898
2006-11-03,1.915963,1.974556,1.915963,1.968696,0.151965,0.077421
2006-11-06,1.951119,1.980415,1.945260,1.980415,0.078130,0.039873
2006-11-07,1.980415,2.009711,1.974556,1.992133,0.111856,0.058165
2006-11-08,1.992133,1.992133,1.962837,1.968696,0.055527,0.028227


In [68]:
f.predict(calc_data)

array([[1.98648779],
       [1.9411367 ],
       [1.94736523],
       ...,
       [5.39444372],
       [5.4349842 ],
       [5.45478917]])